# Zero to JupyterHub Instructions

Most everything here is cribbed from <https://zero-to-jupyterhub.readthedocs.io/en/latest/amazon/step-zero-aws.html>, but I wanted to record my steps in case I ever needed to redo it, or remember certain things.

## Create the kubernetes control node

Create an AWS node with the correct permissions for s3, ec2, iam, etc.

## SSH into the kubernetes control node

Source the following:

```sh
#!/bin/bash

export NAMESPACE="n3f-jupyter"
export NAME="${NAMESPACE}.k8s.local"
export KOPS_STATE_STORE="s3://work.because.jupyter.config"
export REGION=`curl -s http://169.254.169.254/latest/dynamic/instance-identity/document|grep region|awk -F\" '{print $4}'`
export ZONES=$(aws ec2 describe-availability-zones --region $REGION | grep ZoneName | awk '{print $2}' | tr -d '"')
export ZONES=$(echo $ZONES | tr -d " " | rev | cut -c 2- | rev)
```

# Create the Kubernetes cluster

```sh
$ kops create cluster $NAME   --zones=$ZONES   --authorization=RBAC   --master-size=t2.micro   --master-volume-size=10   --node-size=t2.medium   --node-volume-size=30  --yes
```

That will return after spitting out a ton of information, but you can check the status by running this:

```sh
$ time until kops validate cluster; do sleep 15 ; done
```

Then verify that the nodes are good by running:

```sh
$ kubectl get node
```

# The post cluster create steps

I placed these in another shell script, `post_cluster_create.sh`

```sh
#!/bin/bash

kubectl apply -f storageclass.yml
kubectl --namespace kube-system create serviceaccount tiller
kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
helm init --service-account tiller
kubectl --namespace=kube-system patch deployment tiller-deploy --type=json --patch='[{"op": "add", "path": "/spec/template/spec/containers/0/command", "value": ["/tiller", "--listen=localhost:44134"]}]'
```

If you haven't installed the helm jupyterhub repo, do it with `helm repo install https://jupyterhub.github.io/helm-chart/`

# Helm install

## `config.yml`

```yml
proxy:
  secretToken: "c00cbe3b0e007c5508971cd8da224e0c52703db9cd2841b012a2f384e68b5951"
#prePuller:
#  hook:
#    enabled: false
auth:
  type: github
  github:
    clientId: "ee753bcb95826a8928b3"
    clientSecret: "aa004ca00d9849098a0fa72b1515329d5495e6f8"
    callbackUrl: "https://python.because.work/hub/oauth_callback"
singleuser:
  image:
#    name: jupyter/scipy-notebook
#    tag: latest
    name: brentnef/intro-python
    tag: v2
  lifecycleHooks:
    postStart:
      exec:
        command: ["gitpuller", "https://github.com/n3f/intro-python", "session01", "session01"]
```

## Actual command

```sh
$ helm install jupyterhub/jupyterhub     --version=0.7.0-beta.2     --name=$NAMESPACE     --namespace=$NAMESPACE -f config.yml --timeout 10000 --wait
```

## Get the IP/address to connect to

```sh
$ kubectl --namespace=$NAMESPACE describe svc proxy-public
```

## Upgrade command

```sh
$ helm upgrade $NAMESPACE jupyterhub/jupyterhub --version=0.7.0-beta.2  -f config.yml --timeout=10000 --wait
```

## Troubleshooting

### Get the pod listings

```sh
$ kubectl --namespace=$NAMESPACE get pod
```

### Query pod

```sh
$ kubectl describe pod -n $NAMESPACE proxy-6c76fc64d8-xhrjp
```

## SSL

There seems to be a bug with the kubernetes creation of the stack, so when the helm process runs, you need to manually add the SSL connection to the load balancer ***and*** the security group.